Importing necessary packages, i.e. Spacy

In [338]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler

Loading the english nlp model and adding a named entity recognition (ner) pipeline

In [339]:
nlp = spacy.load("en_core_web_sm")
ner=nlp.get_pipe("ner")

Ceating an EntityRuler pipeline

In [340]:
ruler = EntityRuler(nlp, overwrite_ents=True)

Loading in list of strings to be used for final testing.

In [341]:
og_strings = ["MARKS AND SPENCER LTD", "LONDON","ICNAO02312",
               "LONDON, GREAT BRITAIN", "TOYS", "INTEL LLC", 
               "M&S CORPORATION Limited", "LONDON, ENGLAND"]
string_list = []
for i in og_strings:
  mod_str = i.lower()
  string_list.append(mod_str)
print(string_list)

['marks and spencer ltd', 'london', 'icnao02312', 'london, great britain', 'toys', 'intel llc', 'm&s corporation limited', 'london, england']


Viewing entity labels currently in the NER pipe:

In [342]:
labels = ner.labels
print(labels)

('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')


If we test the NER before training, we'll see that it doesn't find our entity:

In [343]:
doc = nlp(string_list[2]) 

Adding new entity label for address. Probably need lots more training examples for addresses for the model to be accurate.

In [344]:
LABELS = ["ADDRESS"]
#ner.add_label(LABEL)
for i in LABELS:
  ner.add_label(i)

Creating training data for the NER, which is in the format accepted by Spacy.

In [345]:
TRAIN_DATA =[
             ("An order from marks and spencer limited", {
                 'entities': [(14, 39, 'ORG')] 
             }),
             ("Accounts for m&s", {
                 'entities': [(13, 16, 'ORG')] 
             }),
             ("Return for m&s limited", {
                 'entities': [(11, 22, 'ORG')] 
             }),
             ("Return for marks and spencer plc", {
                 'entities': [(11, 32, 'ORG')] 
             }),
             ("An order from nvidia", {
                 'entities': [(14, 20, 'ORG')] 
             }),
             ("Accounts for nvidia ireland", {
                 'entities': [(13, 27, 'ORG')] 
             }),
             ("Accounts for nvidia america", {
                 'entities': [(13, 27, 'ORG')] 
             }),
             ("Accounts for nvidia corporation", {
                 'entities': [(13, 31, 'ORG')] 
             }),
             ("Accounts for nvidia corp.", {
                 'entities': [(13, 25, 'ORG')] 
             }),
             ("An order from intel", {
                 'entities': [(14, 19, 'ORG')] 
             }),
             ("Accounts for intel corporation", {
                 'entities': [(13, 30, 'ORG')] 
             }),
              ("apple", {
                 'entities': [(0, 5, 'ORG')]
             }),
             ("An order from apple", {
                 'entities': [(14, 19, 'ORG')] 
             }),
             ("Accounts for jl partnership", {
                 'entities': [(13, 27, 'ORG')] 
             }),
             ("An order from john lewis partnership", {
                 'entities': [(14, 36, 'ORG')] 
             }),
             ("Accounts for john lewis", {
                 'entities': [(13, 23, 'ORG')] 
             }),
             ("london, eng", {
                 'entities': [(0, 11, 'LOC')] 
             }),
             ("london", {
                 'entities': [(0, 6, 'LOC')] 
             }),
             ("london, uk", {
                 'entities': [(0, 10, 'LOC')] 
             }),
             ("asia", {
                 'entities': [(0, 4, 'LOC')]
             }),
              ("east asia", {
                 'entities': [(0, 9, 'LOC')]
             }),
              ("south asia", {
                 'entities': [(0, 10, 'LOC')]
             }),
             ("beijing, china", {
                 'entities': [(0, 14, 'LOC')] 
             }),
             ("china", {
                 'entities': [(0, 5, 'LOC')] 
             }),
             ("shanghai, china", {
                 'entities': [(0, 15, 'LOC')] 
             }),
             ("ny, usa", {
                 'entities': [(0, 7, 'LOC')] 
             }),
             ("new york", {
                 'entities': [(0, 8, 'LOC')] 
             }),
             ("sydney, australia", {
                 'entities': [(0, 17, 'LOC')] 
             }),
             ("sydney, aus", {
                 'entities': [(0, 11, 'LOC')] 
             }),
              ("sydney", {
                 'entities': [(0, 6, 'LOC')] 
             }),
             ("Order for a hardwood table.", {
                 'entities': [(12, 26, 'PRODUCT')] 
             }),
             ("Invoice for a plastic chair", {
                 'entities': [(14, 27, 'PRODUCT')] 
             }),
             ("Invoice for a coffee table.", {
                 'entities': [(14, 26, 'PRODUCT')] 
             }),
             ("An invoice for a new plastic bottle", {
                 'entities': [(21, 35, 'PRODUCT')]
             }),
             ("A fillet knife.", {
                 'entities': [(2, 14, 'PRODUCT')]
             }),
             ("slough, se12 2xy", {
                 'entities': [(0, 16, 'ADDRESS')]
             }),
             ("33 timber yard, london, l1 8xy", {
                 'entities': [(0,30, 'ADDRESS')]
             }),
             ("44 china road, kowloon, hong kong", {
                 'entities': [(0, 33, 'ADDRESS')]
             }),
             ("100 north riverside plaza, chicago, illinois, united states", {
                 'entities': [(0, 58, 'ADDRESS')]
             }),
             ("1 apple park way, cupertino, california, united states", {
                 'entities': [(0, 54, 'ADDRESS')]
             }),
             ("santa clara, california, us", {
                 'entities': [(0, 27, 'ADDRESS')]
             }),
             ("2200 mission college blvd, santa clara, ca", {
                 'entities': [(0, 42, 'ADDRESS')]
             }),
             ("1 hacker way, menlo park, ca", {
                 'entities': [(0, 28, 'ADDRESS')]
             }),

             
]

Initialising nlp ready for training:

In [346]:
optimizer = nlp.resume_training()

Excluding all other pipes during training:

In [347]:
# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

Training the NER for 30 iterations:

In [348]:
# Importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

  sizes = compounding(1.0, 4.0, 1.001)
  # Training for 30 iterations     
  for itn in range(30):
    # shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=sizes)
    # ictionary to store losses
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      # Calling update() over the iteration
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      print("Losses", losses)

Losses {'ner': 6.380213725464554}
Losses {'ner': 10.652604329483719}
Losses {'ner': 16.30599134494704}
Losses {'ner': 18.239293983795584}
Losses {'ner': 18.24222218746589}
Losses {'ner': 20.136048552936778}
Losses {'ner': 24.878168222851023}
Losses {'ner': 31.853354252582072}
Losses {'ner': 33.245430029635905}
Losses {'ner': 43.893124855762004}
Losses {'ner': 46.006511397367}
Losses {'ner': 51.23824853209779}
Losses {'ner': 53.23645443229005}
Losses {'ner': 55.20582074431703}
Losses {'ner': 57.1929019025058}
Losses {'ner': 61.81701086906309}
Losses {'ner': 61.81877038281004}
Losses {'ner': 68.25581439183208}
Losses {'ner': 69.09829975120395}
Losses {'ner': 70.66871465595499}
Losses {'ner': 74.92006589325204}
Losses {'ner': 76.93398508067797}
Losses {'ner': 82.84156491990755}
Losses {'ner': 90.07273339525885}
Losses {'ner': 100.84488522400237}
Losses {'ner': 104.66733031538637}
Losses {'ner': 107.25011365679414}
Losses {'ner': 110.7575196537425}
Losses {'ner': 114.10732429355879}
Losses

Demonstrating the performance of the NER using an example from the initial string list.

In [349]:
doc = nlp(string_list[0]) 
displacy.render(doc, style="ent", jupyter=True)

**Serial Numbers**

I decided to go with a Spacy Matcher for the Serial Numbers due to their repetitive nature that can be easily predicted using regular expressions. 

To finalise the model, this would need to be integrated into the flow of events. Alternatively, we could train the NER to detect the ID numbers as a unique entity and we could check these against an EntityLinker, in the same way as the other entites. 

In [350]:
import re
doc = nlp("XYZ 13423 / ILD, ABC/ICL/20891NC")
#doc = nlp(string_list[2])
print([t.text for t in doc])

serial_num_matches = []

expressions = [r"[A-Za-z]{3,}[/]?[A-Za-z]{3,}[/]?[0-9]{1,}[A-Za-z]+",
               r"[A-Za-z]{3,}[\s]?[/]?[0-9]+[\s]?[/]?[\s]?[A-Za-z]{3,}",
               r"[A-Za-z][0-9]+"]

for expression in expressions:
  for match in re.finditer(expression, doc.text):
      print(match)
      start, end = match.span()
      span = doc.char_span(start, end)
      # This is a Span object or None if match doesn't map to valid token sequence
      if span is not None:
          print("Found match:", span.text)
          span_text = span.text
          string_a = span_text.replace("/"," ")
          print(string_a)
          string_b = string_a.replace(" ","")
          print("replaced string:")
          print(string_b)
          if string_b not in serial_num_matches:
              serial_num_matches.append(string_b)
print(serial_num_matches)

['XYZ', '13423', '/', 'ILD', ',', 'ABC', '/', 'ICL/20891NC']
<re.Match object; span=(17, 32), match='ABC/ICL/20891NC'>
Found match: ABC/ICL/20891NC
ABC ICL 20891NC
replaced string:
ABCICL20891NC
<re.Match object; span=(0, 15), match='XYZ 13423 / ILD'>
Found match: XYZ 13423 / ILD
XYZ 13423   ILD
replaced string:
XYZ13423ILD
['ABCICL20891NC', 'XYZ13423ILD']


# Entity Linking

This next step focuses on the alphanumeric entities that have been trained previously using the EntityRuler. 

We're going to setup an EntityLinker which allows us to match entities with unique ID's and also teaches the model to decipher between synonyms of the same entity. 
The below EntityLinker for example, will match the entities "M&S" and "Marks and Spencer" with the entity "Marks & Spencer" with ID code Q714491. 

In [351]:
import csv
from pathlib import Path

def load_entities():
    entities_loc = Path.cwd().parent / "/entities_aliases.csv"  # distributed alongside this notebook

    names = dict()
    descriptions = dict()
    aliases = dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            qid = row[0]
            print(qid)
            name = row[1]
            print(name)
            desc = row[2]
            aliases_list = row[3:]
            print(aliases_list)
            names[qid] = name
            descriptions[qid] = desc
            aliases[qid] = aliases_list
    return names, descriptions, aliases

Adding entity aliases from file. 

In [352]:
name_dict, desc_dict, alias_dict = load_entities()
for QID in name_dict.keys():
    print(f"{QID}, name={name_dict[QID]}, desc={desc_dict[QID]}, aliases={alias_dict[QID]}")



Q714491
Marks & Spencer
['m&s', 'marks and spencer limited', 'marks and spencer ltd', 'marks and spencer', 'm&s corporation limited']
Q182477
Nvidia
['nvidia ireland', 'nvidia corporation']
Q248
Intel
['intel corp', 'intel corporation', 'integrated electronics', 'intel llc']
Q84
London
['london', 'london, great britain', 'london, england']
Q48
Asia
['east asia', 'south asia']
P1
hardwood table
['wooden table', 'wood table']
P2
plastic chair
[]
P3
coffee table
['side table']
P4
plastic bottle
['single use bottle']
P5
fillet knife
['fillet blade']
P6
toy
['toys']
Q714491, name=Marks & Spencer, desc=retail company, aliases=['m&s', 'marks and spencer limited', 'marks and spencer ltd', 'marks and spencer', 'm&s corporation limited']
Q182477, name=Nvidia, desc=American multinational technology company, aliases=['nvidia ireland', 'nvidia corporation']
Q248, name=Intel, desc=American semiconductor chip manufacturer, aliases=['intel corp', 'intel corporation', 'integrated electronics', 'intel l

Creating a knowledge base:

In [353]:
from spacy.kb import KnowledgeBase
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=96)

Adding entities to knowledge base:

In [354]:
for qid, desc in desc_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=qid, entity_vector=desc_enc, freq=342)   
    # 342 is an arbitrary value here

Adding aliases for each entity, with a probability of 100%. If we had an entity that could potentially be matched with more than 1 alias then we can add these in here with the appropriate probabilities. E.g. if you had 3 entities for 1 alias then the probabilities would be [0.3,0.3,0.3].

In [355]:
for qid, name in name_dict.items():
  print(qid,name)
  kb.add_alias(alias=name, entities=[qid], probabilities=[1])   # 100% prior probability P(entity|alias)

Q714491 Marks & Spencer
Q182477 Nvidia
Q248 Intel
Q84 London
Q48 Asia
P1 hardwood table
P2 plastic chair
P3 coffee table
P4 plastic bottle
P5 fillet knife
P6 toy


Adding Aliases for each unique cluster.

In [356]:
#print(alias_dict)
#print(alias_dict.values())

for qid, alias in alias_dict.items():
  for synm in alias:
    print("Alias no: ", qid)
    print("Alias: ", synm)
    print("")

    kb.add_alias(alias=synm, entities=[qid], probabilities=[1])  # sum([probs]) should be <= 1 !

Alias no:  Q714491
Alias:  m&s

Alias no:  Q714491
Alias:  marks and spencer limited

Alias no:  Q714491
Alias:  marks and spencer ltd

Alias no:  Q714491
Alias:  marks and spencer

Alias no:  Q714491
Alias:  m&s corporation limited

Alias no:  Q182477
Alias:  nvidia ireland

Alias no:  Q182477
Alias:  nvidia corporation

Alias no:  Q248
Alias:  intel corp

Alias no:  Q248
Alias:  intel corporation

Alias no:  Q248
Alias:  integrated electronics

Alias no:  Q248
Alias:  intel llc

Alias no:  Q84
Alias:  london

Alias no:  Q84
Alias:  london, great britain

Alias no:  Q84
Alias:  london, england

Alias no:  Q48
Alias:  east asia

Alias no:  Q48
Alias:  south asia

Alias no:  P1
Alias:  wooden table

Alias no:  P1
Alias:  wood table

Alias no:  P3
Alias:  side table

Alias no:  P4
Alias:  single use bottle

Alias no:  P5
Alias:  fillet blade

Alias no:  P6
Alias:  toys



In [357]:
print(name_dict)

{'Q714491': 'Marks & Spencer', 'Q182477': 'Nvidia', 'Q248': 'Intel', 'Q84': 'London', 'Q48': 'Asia', 'P1': 'hardwood table', 'P2': 'plastic chair', 'P3': 'coffee table', 'P4': 'plastic bottle', 'P5': 'fillet knife', 'P6': 'toy'}


Taking a look at the different cluster ID's. 

In [358]:
qids = name_dict.keys()
for qid in qids:
  print(qid)

Q714491
Q182477
Q248
Q84
Q48
P1
P2
P3
P4
P5
P6


In [359]:
print(len(kb))
print(kb.get_entity_strings())

11
['Q248', 'P1', 'P2', 'Q714491', 'P6', 'P5', 'Q48', 'Q84', 'P4', 'Q182477', 'P3']


In [360]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

Entities in the KB: ['Q248', 'P1', 'P2', 'Q714491', 'P6', 'P5', 'Q48', 'Q84', 'P4', 'Q182477', 'P3']
Aliases in the KB: ['intel llc', 'nvidia corporation', 'Nvidia', 'east asia', 'london', 'Asia', 'coffee table', 'm&s corporation limited', 'London', 'south asia', 'integrated electronics', 'wooden table', 'london, england', 'm&s', 'toy', 'plastic chair', 'intel corp', 'single use bottle', 'Marks & Spencer', 'fillet knife', 'Intel', 'marks and spencer ltd', 'toys', 'intel corporation', 'fillet blade', 'marks and spencer limited', 'marks and spencer', 'nvidia ireland', 'side table', 'hardwood table', 'plastic bottle', 'wood table', 'london, great britain']


Checking some of the ID candidates for  entity strings the EntityLinker expects, and one extra entity that it won't recognise.

In [361]:
print(f"Candidates for 'London': {[c.entity_ for c in kb.get_candidates('London')]}")
print(f"Candidates for 'plastic chair': {[c.entity_ for c in kb.get_candidates('plastic chair')]}")
print(f"Candidates for 'Intel': {[c.entity_ for c in kb.get_candidates('Intel')]}")
print(f"Candidates for 'Asia': {[c.entity_ for c in kb.get_candidates('Asia')]}")
print(f"Candidates for 'Australia': {[c.entity_ for c in kb.get_candidates('Australia')]}")


Candidates for 'London': ['Q84']
Candidates for 'plastic chair': ['P2']
Candidates for 'Intel': ['Q248']
Candidates for 'Asia': ['Q48']
Candidates for 'Australia': []


Saving knowledge base and nlp model to disk.

In [362]:
import os
output_dir = Path.cwd().parent / "my_output"
if not os.path.exists(output_dir):
    os.mkdir(output_dir) 
kb.dump(output_dir / "my_kb")

In [363]:
nlp.to_disk(output_dir / "my_nlp")

Loading in manual annotations. This could be done alternatively by using a custom Prodigy recipe.

1. Loading company annotations

In [364]:
annotations_loc = Path.cwd().parent / "/company_input.csv"

dataset = []
with annotations_loc.open("r", encoding="utf8") as annofile:
    csvreader = csv.reader(annofile, delimiter=",")
    for row in csvreader:
        #print(row)
        text = row[0]
        answer = row[6]
        if answer == "accept":
            QID = row[5]
            offset = (int(row[1]), int(row[2]))
            links_dict = {QID: 1.0}
        dataset.append((text, {"links": {offset: links_dict}}))

for i in range(len(dataset)):
  print(dataset[i])

('marks and spencer limited', {'links': {(0, 25): {'Q714491': 1.0}}})
('marks and spencer', {'links': {(0, 17): {'Q714491': 1.0}}})
('marks and spencer ltd', {'links': {(0, 21): {'Q714491': 1.0}}})
('marks and spencer corporation limited', {'links': {(0, 33): {'Q714491': 1.0}}})
('nvidia', {'links': {(0, 6): {'Q182477': 1.0}}})
('nvidia ireland', {'links': {(0, 14): {'Q182477': 1.0}}})
('intel corp', {'links': {(0, 10): {'Q248': 1.0}}})
('intel corporation', {'links': {(0, 17): {'Q248': 1.0}}})
('intel llc', {'links': {(0, 9): {'Q248': 1.0}}})


2. Loading location annotations

In [365]:
location_annotations_loc = Path.cwd().parent / "/locations_input.csv"  # distributed alongside this notebook

with location_annotations_loc.open("r", encoding="utf8") as annofile:
    csvreader = csv.reader(annofile, delimiter=",")
    for row in csvreader:
        #print(row)
        text = row[0]
        answer = row[6]
        if answer == "accept":
            QID = row[5]
            offset = (int(row[1]), int(row[2]))
            links_dict = {QID: 1.0}
        dataset.append((text, {"links": {offset: links_dict}}))

for i in range(len(dataset)):
  print(dataset[i])

('marks and spencer limited', {'links': {(0, 25): {'Q714491': 1.0}}})
('marks and spencer', {'links': {(0, 17): {'Q714491': 1.0}}})
('marks and spencer ltd', {'links': {(0, 21): {'Q714491': 1.0}}})
('marks and spencer corporation limited', {'links': {(0, 33): {'Q714491': 1.0}}})
('nvidia', {'links': {(0, 6): {'Q182477': 1.0}}})
('nvidia ireland', {'links': {(0, 14): {'Q182477': 1.0}}})
('intel corp', {'links': {(0, 10): {'Q248': 1.0}}})
('intel corporation', {'links': {(0, 17): {'Q248': 1.0}}})
('intel llc', {'links': {(0, 9): {'Q248': 1.0}}})
('london', {'links': {(0, 6): {'Q84': 1.0}}})
('london, england', {'links': {(0, 15): {'Q84': 1.0}}})
('london, great britain', {'links': {(0, 21): {'Q84': 1.0}}})
('asia', {'links': {(0, 4): {'Q48': 1.0}}})
('south asia', {'links': {(0, 10): {'Q48': 1.0}}})
('east asia', {'links': {(0, 9): {'Q48': 1.0}}})


3. Loading in product annotations

In [366]:
product_annotations_loc = Path.cwd().parent / "/product_input.csv"  # distributed alongside this notebook

with product_annotations_loc.open("r", encoding="utf8") as annofile:
    csvreader = csv.reader(annofile, delimiter=",")
    for row in csvreader:
        #print(row)
        text = row[0]
        answer = row[6]
        if answer == "accept":
            QID = row[5]
            offset = (int(row[1]), int(row[2]))
            links_dict = {QID: 1.0}
        dataset.append((text, {"links": {offset: links_dict}}))

for i in range(len(dataset)):
  print(dataset[i])

('marks and spencer limited', {'links': {(0, 25): {'Q714491': 1.0}}})
('marks and spencer', {'links': {(0, 17): {'Q714491': 1.0}}})
('marks and spencer ltd', {'links': {(0, 21): {'Q714491': 1.0}}})
('marks and spencer corporation limited', {'links': {(0, 33): {'Q714491': 1.0}}})
('nvidia', {'links': {(0, 6): {'Q182477': 1.0}}})
('nvidia ireland', {'links': {(0, 14): {'Q182477': 1.0}}})
('intel corp', {'links': {(0, 10): {'Q248': 1.0}}})
('intel corporation', {'links': {(0, 17): {'Q248': 1.0}}})
('intel llc', {'links': {(0, 9): {'Q248': 1.0}}})
('london', {'links': {(0, 6): {'Q84': 1.0}}})
('london, england', {'links': {(0, 15): {'Q84': 1.0}}})
('london, great britain', {'links': {(0, 21): {'Q84': 1.0}}})
('asia', {'links': {(0, 4): {'Q48': 1.0}}})
('south asia', {'links': {(0, 10): {'Q48': 1.0}}})
('east asia', {'links': {(0, 9): {'Q48': 1.0}}})
('hardwood table', {'links': {(0, 14): {'P1': 1.0}}})
('plastic chair', {'links': {(0, 13): {'P2': 1.0}}})
('coffee table', {'links': {(0, 12)

Setting the "gold" ID's and counting how many are in each category.

In [367]:
gold_ids = []
for text, annot in dataset:
    for span, links_dict in annot["links"].items():
        for link, value in links_dict.items():
            if value:
                gold_ids.append(link)

from collections import Counter
print(Counter(gold_ids))

Counter({'Q714491': 4, 'Q248': 3, 'Q84': 3, 'Q48': 3, 'Q182477': 2, 'P1': 1, 'P2': 1, 'P3': 1, 'P4': 1, 'P5': 1, 'P6': 1})


Creating test and train datasets. Given the largest category has 3 unique ID's, 2/3's of the ID's will be the train set, and 1/3 will be the test. For the categories with only 1 unique value, this means there is no test for these particular ID's. This could easily be improved by adding more aliases for the undersampled ID. Without accounting for the unequal distribution of ID's, we'll likely encounter overfitting within the model.

In [368]:
import random

train_dataset = []
test_dataset = []
print(qids)
for QID in qids:
    indices = [i for i, j in enumerate(gold_ids) if j == QID]
    len(indices)
    train_dataset.extend(dataset[index] for index in indices[0:2])
    print(train_dataset)  # first 8 in training
    test_dataset.extend(dataset[index] for index in indices[2:3])
    print(test_dataset)
    # last 2 in test
    
random.shuffle(train_dataset)
random.shuffle(test_dataset)

dict_keys(['Q714491', 'Q182477', 'Q248', 'Q84', 'Q48', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6'])
[('marks and spencer limited', {'links': {(0, 25): {'Q714491': 1.0}}}), ('marks and spencer', {'links': {(0, 17): {'Q714491': 1.0}}})]
[('marks and spencer ltd', {'links': {(0, 21): {'Q714491': 1.0}}})]
[('marks and spencer limited', {'links': {(0, 25): {'Q714491': 1.0}}}), ('marks and spencer', {'links': {(0, 17): {'Q714491': 1.0}}}), ('nvidia', {'links': {(0, 6): {'Q182477': 1.0}}}), ('nvidia ireland', {'links': {(0, 14): {'Q182477': 1.0}}})]
[('marks and spencer ltd', {'links': {(0, 21): {'Q714491': 1.0}}})]
[('marks and spencer limited', {'links': {(0, 25): {'Q714491': 1.0}}}), ('marks and spencer', {'links': {(0, 17): {'Q714491': 1.0}}}), ('nvidia', {'links': {(0, 6): {'Q182477': 1.0}}}), ('nvidia ireland', {'links': {(0, 14): {'Q182477': 1.0}}}), ('intel corp', {'links': {(0, 10): {'Q248': 1.0}}}), ('intel corporation', {'links': {(0, 17): {'Q248': 1.0}}})]
[('marks and spencer ltd', {'lin

Creating train dataset based on nlp entity annotations.

In [369]:
TRAIN_DOCS = []
for text, annotation in train_dataset:
    doc = nlp(text)     # to make this more efficient, you can use nlp.pipe() just once for all the texts
    TRAIN_DOCS.append((doc, annotation))

In [370]:
print(TRAIN_DOCS)
print("")
print(test_dataset)

[(coffee table, {'links': {(0, 12): {'P3': 1.0}}}), (nvidia, {'links': {(0, 6): {'Q182477': 1.0}}}), (fillet knife, {'links': {(0, 12): {'P5': 1.0}}}), (asia, {'links': {(0, 4): {'Q48': 1.0}}}), (hardwood table, {'links': {(0, 14): {'P1': 1.0}}}), (plastic chair, {'links': {(0, 13): {'P2': 1.0}}}), (toys, {'links': {(0, 4): {'P6': 1.0}}}), (plastic bottle, {'links': {(0, 14): {'P4': 1.0}}}), (south asia, {'links': {(0, 10): {'Q48': 1.0}}}), (marks and spencer, {'links': {(0, 17): {'Q714491': 1.0}}}), (intel corp, {'links': {(0, 10): {'Q248': 1.0}}}), (london, england, {'links': {(0, 15): {'Q84': 1.0}}}), (nvidia ireland, {'links': {(0, 14): {'Q182477': 1.0}}}), (london, {'links': {(0, 6): {'Q84': 1.0}}}), (marks and spencer limited, {'links': {(0, 25): {'Q714491': 1.0}}}), (intel corporation, {'links': {(0, 17): {'Q248': 1.0}}})]

[('intel llc', {'links': {(0, 9): {'Q248': 1.0}}}), ('london, great britain', {'links': {(0, 21): {'Q84': 1.0}}}), ('marks and spencer ltd', {'links': {(0, 2

Creating an EntityLinker and setting the knowledge base to the one that's just been created. Adding the entity linker to the nlp. 

In [371]:
entity_linker = nlp.create_pipe("entity_linker", config={"incl_prior": True})
entity_linker.set_kb(kb)
nlp.add_pipe(entity_linker, last=True)

In [372]:
entity_linker.set_kb(kb)

Training the EntityLinker for 500 iterations with the training dataset which has been pre-formatted. 

In [373]:
from spacy.util import minibatch, compounding

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "entity_linker"]
with nlp.disable_pipes(*other_pipes):   # train only the entity_linker
    optimizer = nlp.begin_training()
    for itn in range(500):   # 500 iterations takes about a minute to train
        random.shuffle(TRAIN_DOCS)
        batches = minibatch(TRAIN_DOCS, size=compounding(1.0, 4.0, 1.001))  # increasing batch sizes
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
        if itn % 50 == 0:
            print(itn, "Losses", losses)   # print the training loss
print(itn, "Losses", losses)

0 Losses {'entity_linker': 13.338062703609467}
50 Losses {'entity_linker': 0.8009167313575745}
100 Losses {'entity_linker': 0.42621612548828125}
150 Losses {'entity_linker': 0.38456714153289795}
200 Losses {'entity_linker': 0.19490379095077515}
250 Losses {'entity_linker': 0.19539499282836914}
300 Losses {'entity_linker': 0.16055577993392944}
350 Losses {'entity_linker': 0.15167778730392456}
400 Losses {'entity_linker': 0.10390567779541016}
450 Losses {'entity_linker': 0.08823776245117188}
499 Losses {'entity_linker': 0.06734639406204224}


Doing an initial test with the list of strings provided at the very beginning. 

In [374]:
for i in string_list:
    doc = nlp(i)
    for ent in doc.ents:
        print(ent.text, ent.label_, ent.kb_id_)

marks and spencer ltd ORG Q714491
london LOC Q84
icnao02312 ORG NIL
london, great LOC NIL
toys LOC P6
intel llc LOC Q248
london, england LOC Q84


In [375]:
print(test_dataset)

[('intel llc', {'links': {(0, 9): {'Q248': 1.0}}}), ('london, great britain', {'links': {(0, 21): {'Q84': 1.0}}}), ('marks and spencer ltd', {'links': {(0, 21): {'Q714491': 1.0}}}), ('east asia', {'links': {(0, 9): {'Q48': 1.0}}})]


# Testing on the final test dataset.

Creating a dictionary of lists for the final unique entity clusters. 

Each key corresponds to a unique entity (e.g. Marks and Spencer), and the value corresponds to a list of all the aliases found for that entity (e.g. M&S, Marks & Spencer Ltd etc.)

The next code cell is for development purposes and wouldn't be used in the final production ready model.

In [386]:
unique_entity_clusters = {} 
for text, true_annot in test_dataset:
    print("String ", text)
    print(f"Gold annotation: {true_annot}")
    doc = nlp(text)  # to make this more efficient, you can use nlp.pipe() just once for all the texts
    for ent in doc.ents:
        print(f"Prediction: {ent.text}, {ent.label_}, {ent.kb_id_}")
        if ent.kb_id_ in unique_entity_clusters:
            "Unique Cluster exists"
            if ent.text in unique_entity_clusters[ent.kb_id_]:
              print("Entity already exists")
              print("")
            else:
              unique_entity_clusters[ent.kb_id_].append(ent.text)
              print("")
        else:
            print("Unique cluster does not exist.")
            unique_entity_clusters[ent.kb_id_] = [ent.text]
            print("Added unique cluster and entity.")
            print("")

print("The final unique entity clusters are:")
print(unique_entity_clusters)


String  intel llc
Gold annotation: {'links': {(0, 9): {'Q248': 1.0}}}
Prediction: intel llc, LOC, Q248
Unique cluster does not exist.
Added unique cluster and entity.

String  london, great britain
Gold annotation: {'links': {(0, 21): {'Q84': 1.0}}}
Prediction: london, great, LOC, NIL
Unique cluster does not exist.
Added unique cluster and entity.

String  marks and spencer ltd
Gold annotation: {'links': {(0, 21): {'Q714491': 1.0}}}
Prediction: marks and spencer ltd, ORG, Q714491
Unique cluster does not exist.
Added unique cluster and entity.

String  east asia
Gold annotation: {'links': {(0, 9): {'Q48': 1.0}}}
Prediction: east asia, LOC, Q48
Unique cluster does not exist.
Added unique cluster and entity.

The final unique entity clusters are:
{'Q248': ['intel llc'], 'NIL': ['london, great'], 'Q714491': ['marks and spencer ltd'], 'Q48': ['east asia']}


Proof that a new unique is added to the correct cluster if the cluster already exists:

In [387]:
if "Q248" in unique_entity_clusters:
  print("Unique Cluster exists")
  if "intel corp" in unique_entity_clusters["Q248"]:
    print("Entity already exists")
  else:
    unique_entity_clusters["Q248"].append("intel corp")
else:
  print("Unique cluster does not exist.")
  unique_entity_clusters["Q248"] = ["intel corp"]
  print("Added unique cluster and entity.")

print("The final unique entity clusters are:")
print(unique_entity_clusters)

Unique Cluster exists
The final unique entity clusters are:
{'Q248': ['intel llc', 'intel corp'], 'NIL': ['london, great'], 'Q714491': ['marks and spencer ltd'], 'Q48': ['east asia']}


Replicating the above process with the final string. This is the code that would be implmented in production. 

In [388]:
unique_entity_clusters = {} 
for text in string_list:
    print("String ", text)
    #print(f"Gold annotation: {true_annot}")
    doc = nlp(text)  # to make this more efficient, you can use nlp.pipe() just once for all the texts
    for ent in doc.ents:
        print(f"Prediction: {ent.text}, {ent.label_}, {ent.kb_id_}")
        if ent.kb_id_ in unique_entity_clusters:
            "Unique Cluster exists"
            if ent.text in unique_entity_clusters[ent.kb_id_]:
              print("Entity already exists")
              print("")
            else:
              unique_entity_clusters[ent.kb_id_].append(ent.text)
              print("")
        else:
            print("Unique cluster does not exist.")
            unique_entity_clusters[ent.kb_id_] = [ent.text]
            print("Added unique cluster and entity.")
            print("")

print("The final unique entity clusters are:")
print(unique_entity_clusters)

String  marks and spencer ltd
Prediction: marks and spencer ltd, ORG, Q714491
Unique cluster does not exist.
Added unique cluster and entity.

String  london
Prediction: london, LOC, Q84
Unique cluster does not exist.
Added unique cluster and entity.

String  icnao02312
Prediction: icnao02312, ORG, NIL
Unique cluster does not exist.
Added unique cluster and entity.

String  london, great britain
Prediction: london, great, LOC, NIL

String  toys
Prediction: toys, LOC, P6
Unique cluster does not exist.
Added unique cluster and entity.

String  intel llc
Prediction: intel llc, LOC, Q248
Unique cluster does not exist.
Added unique cluster and entity.

String  m&s corporation limited
String  london, england
Prediction: london, england, LOC, Q84

The final unique entity clusters are:
{'Q714491': ['marks and spencer ltd'], 'Q84': ['london', 'london, england'], 'NIL': ['icnao02312', 'london, great'], 'P6': ['toys'], 'Q248': ['intel llc']}
